In [ ]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

# ===================== 可调参数 =====================
excel_path   = "your_dataset.xlsx"
target_col   = "group"          # 二分类 0/1
cv_folds     = 5
random_state = 42
rfe_k        = 10
rf_top_k     = 10
missing_strategy = "median"     # median / mean

# 导出文件名
export_xlsx  = "FeatureSelection_Report.xlsx"
# ===================================================

# （可选）屏蔽 sklearn FutureWarning（不影响结果）
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

# --------------------- 工具函数 ---------------------
def coerce_binary_series(s: pd.Series) -> pd.Series:
    if pd.api.types.is_bool_dtype(s):
        return s.astype(int)
    if pd.api.types.is_numeric_dtype(s):
        return s

    s_str = s.astype(str).str.strip().str.lower()
    mapping = {
        "0": 0, "1": 1,
        "no": 0, "yes": 1,
        "n": 0, "y": 1,
        "false": 0, "true": 1,
        "negative": 0, "positive": 1,
        "neg": 0, "pos": 1,
        "阴性": 0, "阳性": 1,
        "否": 0, "是": 1,
        "无": 0, "有": 1,
    }
    uniq = s_str.dropna().unique()
    hit_ratio = np.mean([u in mapping for u in uniq]) if len(uniq) > 0 else 0.0
    if hit_ratio >= 0.8:
        return s_str.map(mapping)
    return s

def check_binary_target(y: pd.Series) -> np.ndarray:
    if pd.api.types.is_bool_dtype(y):
        y = y.astype(int)

    if not pd.api.types.is_numeric_dtype(y):
        y2 = coerce_binary_series(y)
        if pd.api.types.is_numeric_dtype(y2):
            y = y2
        else:
            raise ValueError("group 不是数值 0/1，也无法自动识别为二分类文本。请先把 group 处理成 0/1。")

    y = pd.to_numeric(y, errors="coerce")
    if y.isna().any():
        raise ValueError("group 含有无法转成数值的值（NaN）。请检查 group 列。")

    uniq = sorted(y.unique().tolist())
    if set(uniq) != {0, 1}:
        raise ValueError(f"group 不是标准二分类 0/1，当前唯一值为：{uniq}")

    return y.astype(int).values

def preprocess_X(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    X = df.drop(columns=[target_col]).copy()

    for col in X.columns:
        X[col] = coerce_binary_series(X[col])

    X = X.apply(pd.to_numeric, errors="coerce")

    if missing_strategy == "mean":
        fill_vals = X.mean(numeric_only=True)
    else:
        fill_vals = X.median(numeric_only=True)

    X = X.fillna(fill_vals)
    X = X.fillna(0)
    return X

# --------------------- 读取数据 ---------------------
df_raw = pd.read_excel(excel_path)
y = check_binary_target(df_raw[target_col])
X = preprocess_X(df_raw, target_col)

# 数据概况
n = len(df_raw)
pos_rate = float(np.mean(y))
missing_rate = df_raw.drop(columns=[target_col]).isna().mean().sort_values(ascending=False)

data_overview = pd.DataFrame({
    "Metric": ["N", "Positive rate (y=1)", "Negative rate (y=0)", "Num features"],
    "Value": [n, pos_rate, 1 - pos_rate, X.shape[1]]
})

feature_list = pd.DataFrame({"Feature": X.columns})

missing_table = missing_rate.reset_index()
missing_table.columns = ["Feature (raw)", "Missing rate (raw)"]

# 标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===================================================
# 1) L1-LogisticCV (AUC)  —— 纯 L1 用 elasticnet + l1_ratios=(1.0,)
# ===================================================
l1_logit_cv = LogisticRegressionCV(
    Cs=10,
    penalty="elasticnet",
    solver="saga",
    l1_ratios=(1.0,),
    cv=cv_folds,
    scoring="roc_auc",
    class_weight="balanced",
    n_jobs=-1,
    max_iter=20000,
    random_state=random_state,
    use_legacy_attributes=True,
)
l1_logit_cv.fit(X_scaled, y)

coef = l1_logit_cv.coef_
coef_vec = coef.ravel() if coef.ndim == 1 else np.max(coef, axis=0)
coef_abs = np.abs(coef).ravel() if coef.ndim == 1 else np.max(np.abs(coef), axis=0)

l1_selected = X.columns[coef_abs > 1e-8]

l1_coef_table = pd.DataFrame({
    "Feature": X.columns,
    "Coef (aggregated)": coef_vec,
    "AbsCoef": coef_abs
}).sort_values("AbsCoef", ascending=False).reset_index(drop=True)

l1_selected_table = l1_coef_table[l1_coef_table["AbsCoef"] > 1e-8].reset_index(drop=True)

# ===================================================
# 2) Random Forest importance
# ===================================================
rf = RandomForestClassifier(
    n_estimators=500,
    random_state=random_state,
    n_jobs=-1,
    class_weight="balanced_subsample"
)
rf.fit(X, y)

rf_importance = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
rf_imp_table = rf_importance.reset_index()
rf_imp_table.columns = ["Feature", "RF importance"]

rf_top_table = rf_imp_table.head(rf_top_k).copy()
rf_selected = rf_top_table["Feature"].tolist()

# ===================================================
# 3) SVM-RFE
# ===================================================
svm = SVC(kernel="linear")
rfe_svm = RFE(estimator=svm, n_features_to_select=rfe_k)
rfe_svm.fit(X_scaled, y)

svm_selected = X.columns[rfe_svm.support_].tolist()
svm_selected_table = pd.DataFrame({"Feature": svm_selected})

# ===================================================
# 4) Logistic-RFE
# ===================================================
log_reg = LogisticRegression(
    solver="liblinear",
    class_weight="balanced",
    random_state=random_state,
    max_iter=10000
)
rfe_logit = RFE(estimator=log_reg, n_features_to_select=rfe_k)
rfe_logit.fit(X_scaled, y)

logit_rfe_selected = X.columns[rfe_logit.support_].tolist()
logit_rfe_selected_table = pd.DataFrame({"Feature": logit_rfe_selected})

# ===================================================
# 5) 综合：交集 & 一致性（≥3）
# ===================================================
method_sets = {
    "L1-Logistic(CV)": set(l1_selected),
    "RF(TopK)": set(rf_selected),
    "SVM-RFE": set(svm_selected),
    "Logistic-RFE": set(logit_rfe_selected),
}

intersection_all4 = sorted(set.intersection(*method_sets.values()))
intersection_table = pd.DataFrame({"Feature": intersection_all4})

all_feats = sorted(set().union(*method_sets.values()))
appear_counts = {f: sum(f in s for s in method_sets.values()) for f in all_feats}

consensus_ge3 = sorted([f for f, c in appear_counts.items() if c >= 3])
consensus_table = pd.DataFrame({"Feature": consensus_ge3})

appear_table = (
    pd.DataFrame({"Feature": all_feats, "Appearances": [appear_counts[f] for f in all_feats]})
      .sort_values(["Appearances", "Feature"], ascending=[False, True])
      .reset_index(drop=True)
)

# 方法-特征矩阵（方便论文附录/审稿问一致性）
matrix = []
for f in all_feats:
    row = {"Feature": f}
    for m, s in method_sets.items():
        row[m] = 1 if f in s else 0
    row["Appearances"] = appear_counts[f]
    matrix.append(row)
method_matrix = pd.DataFrame(matrix).sort_values(["Appearances", "Feature"], ascending=[False, True]).reset_index(drop=True)

# 参数记录
params_table = pd.DataFrame({
    "Parameter": ["excel_path", "target_col", "cv_folds", "random_state", "rfe_k", "rf_top_k", "missing_strategy", "scoring(L1-CV)"],
    "Value": [excel_path, target_col, cv_folds, random_state, rfe_k, rf_top_k, missing_strategy, "roc_auc"]
})

# ===================================================
# 6) 导出到 Excel（多 Sheet）
# ===================================================
with pd.ExcelWriter(export_xlsx, engine="openpyxl") as writer:
    params_table.to_excel(writer, sheet_name="00_Params", index=False)
    data_overview.to_excel(writer, sheet_name="01_DataOverview", index=False)
    feature_list.to_excel(writer, sheet_name="02_FeatureList", index=False)
    missing_table.to_excel(writer, sheet_name="03_MissingRaw", index=False)

    l1_selected_table.to_excel(writer, sheet_name="10_L1_Selected", index=False)
    l1_coef_table.to_excel(writer, sheet_name="11_L1_AllCoefs", index=False)

    rf_top_table.to_excel(writer, sheet_name="20_RF_TopK", index=False)
    rf_imp_table.to_excel(writer, sheet_name="21_RF_AllImportance", index=False)

    svm_selected_table.to_excel(writer, sheet_name="30_SVM_RFE", index=False)
    logit_rfe_selected_table.to_excel(writer, sheet_name="31_Logit_RFE", index=False)

    intersection_table.to_excel(writer, sheet_name="40_Intersection_All4", index=False)
    consensus_table.to_excel(writer, sheet_name="41_Consensus_GE3", index=False)
    appear_table.to_excel(writer, sheet_name="42_AppearCounts", index=False)
    method_matrix.to_excel(writer, sheet_name="43_MethodMatrix", index=False)

print(f"✅ 已导出 Excel 报告：{export_xlsx}")
print("严格交集（All4）:", intersection_all4)
print("一致性≥3:", consensus_ge3)


In [ ]:
import numpy as np
import pandas as pd
import re, json, joblib

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import confusion_matrix, roc_auc_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# ====================== 你的两个特征集（来自 FeatureSelection_Report.xlsx） ======================
primary_features = [
    'your_data'
]

extended_features = [
    'your_data'
]

train_path = "your_dataset.xlsx"
test_path  = "your_dataset.xlsx"
target_col = "group"
RANDOM_STATE = 42

# ====================== 工具函数 ======================
def ensure_binary_01(y: pd.Series, name="target"):
    y_num = pd.to_numeric(y, errors="coerce")
    if y_num.isna().any():
        bad = y[y_num.isna()].unique()
        raise ValueError(f"{name} 含无法转为数值的值：{bad}")
    uniq = sorted(y_num.unique().tolist())
    if set(uniq) != {0, 1}:
        raise ValueError(f"{name} 不是标准二分类0/1，当前唯一值={uniq}")
    return y_num.astype(int).values

def to_numeric_df(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    for c in X.columns:
        X[c] = pd.to_numeric(X[c], errors="coerce")
    return X

def assert_no_missing(X: pd.DataFrame, name="X"):
    if X.isna().any().any():
        miss_cols = X.columns[X.isna().any()].tolist()
        miss_rate = X[miss_cols].isna().mean().sort_values(ascending=False)
        raise ValueError(f"{name} 仍存在缺失值：{miss_rate.to_dict()}")

def calculate_metrics(y_true, y_pred, y_score):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sens = tp/(tp+fn) if (tp+fn)>0 else 0.0
    spec = tn/(tn+fp) if (tn+fp)>0 else 0.0
    acc  = (tp+tn)/(tp+tn+fp+fn)
    ppv  = tp/(tp+fp) if (tp+fp)>0 else 0.0
    npv  = tn/(tn+fn) if (tn+fn)>0 else 0.0
    f1   = 2*tp/(2*tp+fp+fn) if (2*tp+fp+fn)>0 else 0.0
    auc  = roc_auc_score(y_true, y_score) if len(np.unique(y_true))>1 else 0.0
    return {'AUC':auc,'Sensitivity':sens,'Specificity':spec,'Accuracy':acc,'PPV':ppv,'NPV':npv,'F1':f1}

def best_threshold(y_true, y_score, mode='f1'):
    thresholds = np.linspace(0.01, 0.99, 99)
    best_thr, best_score = 0.5, -np.inf
    for t in thresholds:
        y_pred = (y_score >= t).astype(int)
        m = calculate_metrics(y_true, y_pred, y_score)
        score = m['F1'] if mode == 'f1' else (m['Sensitivity'] + m['Specificity'] - 1)
        if score > best_score:
            best_score, best_thr = score, t
    return float(best_thr)

def get_scores(model, X_df):
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X_df)[:, 1]
    if hasattr(model, "decision_function"):
        return model.decision_function(X_df)
    raise ValueError(f"{type(model).__name__} 缺少 predict_proba/decision_function")

def compute_spw(y):
    pos = np.sum(y == 1)
    neg = np.sum(y == 0)
    return max((neg / pos), 1.0) if pos > 0 else 1.0

def oof_scores_fixed_params(base_estimator, best_params, X_df, y_arr, cv):
    """
    用 best_params 固定超参，在开发集全量上做 out-of-fold 预测，返回每个样本的 OOF score
    """
    oof = np.full(len(y_arr), np.nan, dtype=float)
    for tr_idx, va_idx in cv.split(X_df, y_arr):
        m = clone(base_estimator)
        m.set_params(**best_params)

        # XGBoost：scale_pos_weight 建议按fold训练集计算
        if isinstance(m, XGBClassifier):
            m.set_params(scale_pos_weight=compute_spw(y_arr[tr_idx]))

        m.fit(X_df.iloc[tr_idx], y_arr[tr_idx])
        oof[va_idx] = get_scores(m, X_df.iloc[va_idx])

    if np.isnan(oof).any():
        raise RuntimeError("OOF scores 存在 NaN，请检查数据/索引。")
    return oof

# ====================== CV & BayesSearch 设置 ======================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

bayes_common = dict(
    n_iter=30,
    cv=cv,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    optimizer_kwargs={'acq_optimizer': 'sampling'}
)

# ====================== 模型（这里不放 StandardScaler，因为你已经数值化且量纲差异不影响树模型；）
# 对 LR/SVM/KNN 我们用 Pipeline+Scaler，更规范。树模型 passthrough。
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def make_scaler():
    return StandardScaler()

models_config = {
    'Logistic Regression': {
        'estimator': Pipeline([
            ('scaler', make_scaler()),
            ('clf', LogisticRegression(
                random_state=RANDOM_STATE, class_weight='balanced', max_iter=5000
            ))
        ]),
        'search_spaces': {'clf__C': Real(1e-2, 1e2, prior='log-uniform')}
    },

    'Random Forest': {
        'estimator': Pipeline([
            ('scaler', 'passthrough'),
            ('clf', RandomForestClassifier(
                random_state=RANDOM_STATE, class_weight='balanced'
            ))
        ]),
        'search_spaces': {
            'clf__n_estimators': Integer(200, 800),
            'clf__max_depth': Integer(3, 30),
            'clf__min_samples_split': Integer(2, 30),
            'clf__min_samples_leaf': Integer(1, 15),
            'clf__max_features': Categorical(['sqrt', 'log2', None])
        }
    },

    'XGBoost': {
        'estimator': Pipeline([
            ('scaler', 'passthrough'),
            ('clf', XGBClassifier(
                random_state=RANDOM_STATE,
                eval_metric='logloss',
                tree_method='hist'
            ))
        ]),
        'search_spaces': {
            'clf__n_estimators': Integer(200, 1000),
            'clf__max_depth': Integer(2, 10),
            'clf__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
            'clf__subsample': Real(0.6, 1.0),
            'clf__colsample_bytree': Real(0.5, 1.0),
            'clf__reg_lambda': Real(1e-3, 10, prior='log-uniform')
        }
    },

    'SVM': {
        'estimator': Pipeline([
            ('scaler', make_scaler()),
            ('clf', SVC(
                random_state=RANDOM_STATE,
                probability=True,
                class_weight='balanced'
            ))
        ]),
        'search_spaces': [
            {'clf__kernel': Categorical(['linear']),
             'clf__C': Real(1e-2, 1e2, prior='log-uniform')},
            {'clf__kernel': Categorical(['rbf']),
             'clf__C': Real(1e-2, 1e2, prior='log-uniform'),
             'clf__gamma': Real(1e-3, 1, prior='log-uniform')}
        ]
    },

    'KNN': {
        'estimator': Pipeline([
            ('scaler', make_scaler()),
            ('clf', KNeighborsClassifier())
        ]),
        'search_spaces': {
            'clf__n_neighbors': Integer(3, 40),
            'clf__weights': Categorical(['uniform', 'distance']),
            'clf__metric': Categorical(['euclidean', 'manhattan'])
        }
    },

    'Decision Tree': {
        'estimator': Pipeline([
            ('scaler', 'passthrough'),
            ('clf', DecisionTreeClassifier(
                random_state=RANDOM_STATE, class_weight='balanced'
            ))
        ]),
        'search_spaces': {
            'clf__max_depth': Integer(3, 30),
            'clf__min_samples_split': Integer(2, 30),
            'clf__min_samples_leaf': Integer(1, 15),
            'clf__criterion': Categorical(['gini', 'entropy', 'log_loss'])
        }
    }
}

# ====================== 读取数据 ======================
df_train = pd.read_excel(train_path)
df_test  = pd.read_excel(test_path)

y_dev = ensure_binary_01(df_train[target_col], name=target_col)
y_val = ensure_binary_01(df_test[target_col], name=f"{target_col}")

# ====================== 主函数：开发集全量 OOF 选阈值 + 外测 ======================
def run_set(feature_list, tag):
    print(f"\n==================== {tag}（p={len(feature_list)}）====================")

    # 取特征（开发集全量 + val）
    X_dev = df_train[feature_list].copy()
    X_val = df_test[feature_list].copy()

    X_dev = to_numeric_df(X_dev)
    X_val = to_numeric_df(X_val)

    assert_no_missing(X_dev, name=f"X_dev({tag})")
    assert_no_missing(X_val, name=f"X_val({tag})")

    results_oof = []
    results_val = []
    best_models = {}
    best_params_all = {}
    thresholds_f1 = {}

    print("开始 BayesSearch（仅开发集）+ OOF 阈值（开发集内）+  外部验证…")

    for name, cfg in models_config.items():
        print(f"\n🔄 {name} ...")

        est = cfg['estimator']

        # XGBoost：scale_pos_weight 先按全开发集设置（BayesSearch中会在每次fit的训练fold里再被clone覆盖不了）
        # 这里不强制，后面 OOF 和 final fit 会按fold/全量再设置一次。
        if name == 'XGBoost':
            est = clone(est)
            est.set_params(clf__scale_pos_weight=compute_spw(y_dev))

        bayes = BayesSearchCV(
            estimator=est,
            search_spaces=cfg['search_spaces'],
            **bayes_common
        )
        bayes.fit(X_dev, y_dev)

        best_params = bayes.best_params_
        best_params_all[name] = best_params

        # --------- OOF scores（用 best_params 固定）---------
        oof = oof_scores_fixed_params(
            base_estimator=cfg['estimator'],
            best_params=best_params,
            X_df=X_dev,
            y_arr=y_dev,
            cv=cv
        )
        thr = best_threshold(y_dev, oof, mode='f1')
        thresholds_f1[name] = thr

        oof_pred = (oof >= thr).astype(int)
        m_oof = calculate_metrics(y_dev, oof_pred, oof)
        results_oof.append({'Model': name, 'Thr@F1(OOF)': thr, **m_oof})

        # --------- Final model：用全开发集拟合（固定 best_params）---------
        final_model = clone(cfg['estimator'])
        final_model.set_params(**best_params)

        if name == 'XGBoost':
            final_model.set_params(clf__scale_pos_weight=compute_spw(y_dev))

        final_model.fit(X_dev, y_dev)
        best_models[name] = final_model

        # --------- external validation（固定阈值）---------
        te_score = get_scores(final_model, X_val)
        te_pred = (te_score >= thr).astype(int)
        m_te = calculate_metrics(y_val, te_pred, te_score)
        results_val.append({'Model': name, 'Thr@F1(OOF)': thr, **m_te})

        print(f"✅ OOF AUC={m_oof['AUC']:.4f}, Thr={thr:.2f} | val AUC={m_te['AUC']:.4f}, F1={m_te['F1']:.4f}")

    # 汇总 + 保存
    oof_df = pd.DataFrame(results_oof).sort_values('AUC', ascending=False).reset_index(drop=True)
    te_df  = pd.DataFrame(results_val).sort_values('AUC', ascending=False).reset_index(drop=True)

    out_xlsx = f"model_performance_{tag}_OOF_TRIPOD.xlsx"
    with pd.ExcelWriter(out_xlsx) as writer:
        oof_df.to_excel(writer, sheet_name='DEV_OOF_f1', index=False)
        te_df.to_excel(writer, sheet_name='TEST2024_fixedThr', index=False)
        pd.DataFrame(best_params_all).T.to_excel(writer, sheet_name='best_params')
        pd.DataFrame({'Model': list(thresholds_f1.keys()),
                      'Thr@F1(OOF)': list(thresholds_f1.values())}).to_excel(writer, sheet_name='thresholds_f1', index=False)

    for model_name, model in best_models.items():
        slug = re.sub(r'[^A-Za-z0-9_-]+', '_', model_name.strip()).lower()
        joblib.dump(model, f"best_model_{tag}_{slug}.pkl")

    with open(f'best_model_thresholds_f1_{tag}_OOF_TRIPOD.json', 'w', encoding='utf-8') as f:
        json.dump({k: float(v) for k, v in thresholds_f1.items()}, f, ensure_ascii=False, indent=2)

    print(f"\n✅ 完成：{out_xlsx}、best_model_{tag}_*.pkl、best_model_thresholds_f1_{tag}_OOF_TRIPOD.json")
    return oof_df, te_df


# ====================== 跑两套特征集 ======================
run_set(primary_features, tag="primary_5")
run_set(extended_features, tag="extended_8")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# === 配置 ===
MODEL_PKL = "best_model_primary_5_random_forest.pkl"
DEV_XLSX = "your_dataset.xlsx"
TEST_XLSX = "your_dataset.xlsx"
TARGET = "group"

FEATURES = [
    'your_data'
]

# === 读取数据 ===
df_dev = pd.read_excel(DEV_XLSX)
df_test = pd.read_excel(TEST_XLSX)

X_dev = df_dev[FEATURES].apply(pd.to_numeric)
y_dev = df_dev[TARGET].values

X_test = df_test[FEATURES].apply(pd.to_numeric)
y_test = df_test[TARGET].values

model = joblib.load(MODEL_PKL)

# === OOF ROC (Development) ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(y_dev))

for train_idx, val_idx in cv.split(X_dev, y_dev):
    m = model
    m.fit(X_dev.iloc[train_idx], y_dev[train_idx])
    oof[val_idx] = m.predict_proba(X_dev.iloc[val_idx])[:,1]

fpr_dev, tpr_dev, _ = roc_curve(y_dev, oof)
auc_dev = roc_auc_score(y_dev, oof)

# ===  ROC ===
y_test_prob = model.predict_proba(X_test)[:,1]
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_prob)
auc_test = roc_auc_score(y_test, y_test_prob)

# === 画图 ===
plt.figure()
plt.plot(fpr_dev, tpr_dev, label=f'Development (AUC={auc_dev:.3f})')
plt.plot(fpr_test, tpr_test, label=f'Validation (AUC={auc_test:.3f})')
plt.plot([0,1],[0,1],'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.tight_layout()
plt.savefig("Figure_ROC.tiff", dpi=600)
plt.savefig("Figure_ROC.png", dpi=600)
plt.close()

print("ROC figure saved.")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import brier_score_loss
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression

# ===================== 配置 =====================
MODEL_PKL = "best_model_primary_5_random_forest.pkl"   # 改成你的实际文件名
DEV_XLSX  = "your_dataset.xlsx"
TEST_XLSX = "your_dataset.xlsx"
TARGET_COL = "group"
RANDOM_STATE = 42

FEATURES = [
    'your_data'
]

# ===================== 工具函数 =====================
def to_numeric_no_missing(df, cols, name="X"):
    X = df[cols].copy()
    for c in cols:
        X[c] = pd.to_numeric(X[c], errors="coerce")
    if X.isna().any().any():
        miss = X.isna().mean().to_dict()
        raise ValueError(f"{name} still has missing values: {miss}")
    return X

def oof_predict_proba(base_model, X, y, cv):
    oof = np.full(len(y), np.nan, dtype=float)
    for tr, va in cv.split(X, y):
        m = clone(base_model)
        m.fit(X.iloc[tr], y[tr])
        oof[va] = m.predict_proba(X.iloc[va])[:, 1]
    if np.isnan(oof).any():
        raise RuntimeError("OOF probabilities contain NaN.")
    return oof

def calibration_slope_intercept(y, p):
    """
    Fit: logit(P(Y=1)) = intercept + slope * logit(p)
    返回 (intercept, slope)
    """
    eps = 1e-6
    p = np.clip(p, eps, 1 - eps)
    x = np.log(p / (1 - p)).reshape(-1, 1)

    # sklearn 的无惩罚LR：不同版本 sklearn 参数略有差异，下面做兼容
    try:
        lr = LogisticRegression(penalty=None, solver="lbfgs", max_iter=2000)
    except TypeError:
        lr = LogisticRegression(penalty="none", solver="lbfgs", max_iter=2000)

    lr.fit(x, y)
    intercept = float(lr.intercept_[0])
    slope = float(lr.coef_[0][0])
    return intercept, slope

def plot_calibration(y_dev, p_dev, y_test, p_test, n_bins=10, out_prefix="Figure_Calibration"):
    # 分箱校准曲线
    frac_pos_dev, mean_pred_dev = calibration_curve(y_dev, p_dev, n_bins=n_bins, strategy="quantile")
    frac_pos_te,  mean_pred_te  = calibration_curve(y_test, p_test, n_bins=n_bins, strategy="quantile")

    # 画图
    plt.figure()
    plt.plot([0, 1], [0, 1], linestyle="--", linewidth=1)  # 理想线
    plt.plot(mean_pred_dev, frac_pos_dev, marker="o", label="Development (OOF)")
    plt.plot(mean_pred_te,  frac_pos_te,  marker="o", label="validation")

    plt.xlabel("Mean predicted probability")
    plt.ylabel("Observed event rate")
    plt.legend()
    plt.tight_layout()

    plt.savefig(f"{out_prefix}.png", dpi=600, bbox_inches="tight")
    plt.savefig(f"{out_prefix}.tiff", dpi=600, bbox_inches="tight")
    plt.close()

# ===================== 读数据 =====================
df_dev = pd.read_excel(DEV_XLSX)
df_te  = pd.read_excel(TEST_XLSX)

y_dev = pd.to_numeric(df_dev[TARGET_COL], errors="coerce").astype(int).values
y_te  = pd.to_numeric(df_te[TARGET_COL], errors="coerce").astype(int).values

X_dev = to_numeric_no_missing(df_dev, FEATURES, name="X_dev")
X_te  = to_numeric_no_missing(df_te,  FEATURES, name="X_val")

# ===================== 加载模型 =====================
base_model = joblib.load(MODEL_PKL)
if not hasattr(base_model, "predict_proba"):
    raise ValueError("Loaded model does not support predict_proba().")

# ===================== Development：OOF predicted probabilities =====================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
p_dev_oof = oof_predict_proba(base_model, X_dev, y_dev, cv=cv)

# ===================== val：final model fit on full dev =====================
final_model = clone(base_model)
final_model.fit(X_dev, y_dev)
p_te = final_model.predict_proba(X_te)[:, 1]

# ===================== 指标：Brier + slope/intercept =====================
brier_dev = brier_score_loss(y_dev, p_dev_oof)
brier_te  = brier_score_loss(y_te,  p_te)

int_dev, slope_dev = calibration_slope_intercept(y_dev, p_dev_oof)
int_te,  slope_te  = calibration_slope_intercept(y_te,  p_te)

print("\n=== Calibration metrics ===")
print(f"Development (OOF): Brier={brier_dev:.4f}, intercept={int_dev:.3f}, slope={slope_dev:.3f}")
print(f"validation  : Brier={brier_te:.4f}, intercept={int_te:.3f}, slope={slope_te:.3f}")

# ===================== 画图并保存（600 dpi） =====================
plot_calibration(
    y_dev=y_dev, p_dev=p_dev_oof,
    y_test=y_te, p_test=p_te,
    n_bins=10,
    out_prefix="Figure_Calibration_primary5_RF"
)

print("\n✅ Saved: Figure_Calibration_primary5_RF.png / .tiff (600 dpi)")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score

# ===================== 配置 =====================
MODEL_PKL = "best_model_primary_5_random_forest.pkl"   # 按你实际文件名修改
DEV_XLSX  = "your_dataset.xlsx"
TEST_XLSX = "your_dataset.xlsx"
TARGET_COL = "group"
RANDOM_STATE = 42

FEATURES = [
    'your_data'
]

# ===================== 工具函数 =====================
def to_numeric_no_missing(df, cols, name="X"):
    X = df[cols].copy()
    for c in cols:
        X[c] = pd.to_numeric(X[c], errors="coerce")
    if X.isna().any().any():
        miss = X.isna().mean().to_dict()
        raise ValueError(f"{name} still has missing values: {miss}")
    return X

def oof_predict_proba(base_model, X, y, cv):
    oof = np.full(len(y), np.nan, dtype=float)
    for tr, va in cv.split(X, y):
        m = clone(base_model)
        m.fit(X.iloc[tr], y[tr])
        oof[va] = m.predict_proba(X.iloc[va])[:, 1]
    if np.isnan(oof).any():
        raise RuntimeError("OOF probabilities contain NaN.")
    return oof

def bootstrap_auc_ci(y_true, y_score, n_boot=2000, seed=42):
    """
    Bootstrap 95% CI for AUC.
    - 对每次bootstrap重采样（有放回）
    - 若抽样后全为同一类，跳过该次
    """
    rng = np.random.default_rng(seed)
    y_true = np.asarray(y_true)
    y_score = np.asarray(y_score)

    aucs = []
    n = len(y_true)

    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        y_b = y_true[idx]
        s_b = y_score[idx]
        if len(np.unique(y_b)) < 2:
            continue
        aucs.append(roc_auc_score(y_b, s_b))

    if len(aucs) < 200:
        raise RuntimeError(f"Too few valid bootstrap samples: {len(aucs)}")

    aucs = np.sort(np.array(aucs))
    lo = float(np.percentile(aucs, 2.5))
    hi = float(np.percentile(aucs, 97.5))
    return lo, hi

# ===================== 读数据 =====================
df_dev = pd.read_excel(DEV_XLSX)
df_te  = pd.read_excel(TEST_XLSX)

y_dev = pd.to_numeric(df_dev[TARGET_COL], errors="coerce").astype(int).values
y_te  = pd.to_numeric(df_te[TARGET_COL],  errors="coerce").astype(int).values

X_dev = to_numeric_no_missing(df_dev, FEATURES, name="X_dev")
X_te  = to_numeric_no_missing(df_te,  FEATURES, name="X_val")

# ===================== 加载模型 =====================
base_model = joblib.load(MODEL_PKL)
if not hasattr(base_model, "predict_proba"):
    raise ValueError("Loaded model does not support predict_proba().")

# ===================== Development：OOF ROC =====================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
p_dev_oof = oof_predict_proba(base_model, X_dev, y_dev, cv=cv)

auc_dev = roc_auc_score(y_dev, p_dev_oof)
ci_dev = bootstrap_auc_ci(y_dev, p_dev_oof, n_boot=2000, seed=RANDOM_STATE)
fpr_dev, tpr_dev, _ = roc_curve(y_dev, p_dev_oof)

# ===================== val：Final model fit on full dev =====================
final_model = clone(base_model)
final_model.fit(X_dev, y_dev)
p_te = final_model.predict_proba(X_te)[:, 1]

auc_te = roc_auc_score(y_te, p_te)
ci_te = bootstrap_auc_ci(y_te, p_te, n_boot=2000, seed=RANDOM_STATE + 1)
fpr_te, tpr_te, _ = roc_curve(y_te, p_te)

# ===================== 输出数值 =====================
print("\n=== ROC / AUC ===")
print(f"Development (OOF): AUC={auc_dev:.4f} (95% CI {ci_dev[0]:.4f}–{ci_dev[1]:.4f})")
print(f"val validation  : AUC={auc_te:.4f} (95% CI {ci_te[0]:.4f}–{ci_te[1]:.4f})")

# ===================== 画图并保存（600 dpi） =====================
plt.figure()
plt.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

plt.plot(
    fpr_dev, tpr_dev,
    label=f"Development (OOF) AUC {auc_dev:.3f} (95% CI {ci_dev[0]:.3f}–{ci_dev[1]:.3f})"
)
plt.plot(
    fpr_te, tpr_te,
    label=f"val validation AUC {auc_te:.3f} (95% CI {ci_te[0]:.3f}–{ci_te[1]:.3f})"
)

plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.legend(loc="lower right")
plt.tight_layout()

plt.savefig("Figure_ROC_primary5_RF.png", dpi=600, bbox_inches="tight")
plt.savefig("Figure_ROC_primary5_RF.tiff", dpi=600, bbox_inches="tight")
plt.close()

print("\n✅ Saved: Figure_ROC_primary5_RF.png / .tiff (600 dpi)")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold

# ===================== 配置 =====================
MODEL_PKL = "best_model_primary_5_random_forest.pkl"
DEV_XLSX  = "your_dataset.xlsx"
TEST_XLSX = "your_dataset.xlsx"
TARGET_COL = "group"
RANDOM_STATE = 42

FEATURES = [
    'your_data'
]

# ===================== 工具函数 =====================
def to_numeric_no_missing(df, cols):
    X = df[cols].copy()
    for c in cols:
        X[c] = pd.to_numeric(X[c], errors="coerce")
    if X.isna().any().any():
        miss = X.isna().mean().to_dict()
        raise ValueError(f"Missing values detected: {miss}")
    return X

def oof_predict_proba(base_model, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    oof = np.zeros(len(y), dtype=float)
    for tr, va in cv.split(X, y):
        m = clone(base_model)
        m.fit(X.iloc[tr], y[tr])
        oof[va] = m.predict_proba(X.iloc[va])[:, 1]
    return oof

def decision_curve(y_true, prob, thresholds):
    y_true = np.asarray(y_true).astype(int)
    prob = np.asarray(prob).astype(float)

    n = len(y_true)
    prev = np.mean(y_true == 1)

    nb_model = []
    for pt in thresholds:
        # 数值稳定：避免 pt=1 导致爆炸
        if pt <= 0 or pt >= 1:
            nb_model.append(np.nan)
            continue

        pred = (prob >= pt).astype(int)
        tp = np.sum((pred == 1) & (y_true == 1))
        fp = np.sum((pred == 1) & (y_true == 0))

        w = pt / (1 - pt)
        nb = (tp / n) - (fp / n) * w
        nb_model.append(nb)

    nb_model = np.array(nb_model, dtype=float)
    nb_all = prev - (1 - prev) * (thresholds / (1 - thresholds))
    nb_none = np.zeros_like(thresholds, dtype=float)

    return nb_model, nb_all, nb_none

# ===================== 读取数据 =====================
df_dev = pd.read_excel(DEV_XLSX)
df_te  = pd.read_excel(TEST_XLSX)

y_dev = pd.to_numeric(df_dev[TARGET_COL], errors="coerce").astype(int).values
y_te  = pd.to_numeric(df_te[TARGET_COL], errors="coerce").astype(int).values

X_dev = to_numeric_no_missing(df_dev, FEATURES)
X_te  = to_numeric_no_missing(df_te, FEATURES)

# ===================== 加载模型 =====================
base_model = joblib.load(MODEL_PKL)

# ===================== Development OOF =====================
p_dev = oof_predict_proba(base_model, X_dev, y_dev)

# =====================  预测（最终模型：用全开发集拟合后预测） =====================
final_model = clone(base_model)
final_model.fit(X_dev, y_dev)
p_te = final_model.predict_proba(X_te)[:, 1]

# ===================== DCA（关键：阈值范围更合理） =====================
# 建议区间：0.01–0.60，避免 pt/(1-pt) 在右侧爆炸拖垮坐标轴
thresholds = np.arange(0.01, 0.61, 0.01)

nb_dev, nb_all_dev, nb_none = decision_curve(y_dev, p_dev, thresholds)
nb_te,  nb_all_te,  _       = decision_curve(y_te,  p_te,  thresholds)

# ===================== 画图（更杂志友好） =====================
plt.figure(figsize=(7.2, 5.4))

plt.plot(thresholds, nb_dev, label="Development (OOF)")
plt.plot(thresholds, nb_te,  label="2024 validation")

# 只画一个 treat-all（建议用 test），避免图例歧义
plt.plot(thresholds, nb_all_te, linestyle="--", label="Treat all")
plt.plot(thresholds, nb_none,   linestyle="--", label="Treat none")

plt.xlabel("Threshold probability")
plt.ylabel("Net benefit")
plt.xlim(0.0, 0.60)

# 如果仍被极端值压扁，再打开下一行（一般不需要）
# plt.ylim(-0.05, 0.25)

plt.legend(loc="lower left", frameon=True)
plt.tight_layout()

plt.savefig("Figure_DCA_primary5_RF_clean.png", dpi=600, bbox_inches="tight")
plt.savefig("Figure_DCA_primary5_RF_clean.tiff", dpi=600, bbox_inches="tight")
plt.close()

print("✅ DCA figure saved: Figure_DCA_primary5_RF_clean (600 dpi)")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.inspection import permutation_importance

# ===================== 配置 =====================
MODEL_PKL = "best_model_primary_5_random_forest.pkl"
DEV_XLSX = "your_dataset.xlsx"
TARGET_COL = "group"

FEATURES = [
    'your_data'
]

RANDOM_STATE = 42

# ===================== 读取数据 =====================
df = pd.read_excel(DEV_XLSX)

X = df[FEATURES].apply(pd.to_numeric)
y = pd.to_numeric(df[TARGET_COL]).values

# ===================== 加载模型 =====================
model = joblib.load(MODEL_PKL)

# ===================== 计算 Permutation Importance =====================
result = permutation_importance(
    model,
    X,
    y,
    scoring="roc_auc",
    n_repeats=30,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

importances = result.importances_mean
std = result.importances_std

# 排序
indices = np.argsort(importances)

# ===================== 画图 =====================
plt.figure()
plt.barh(range(len(indices)), importances[indices], xerr=std[indices])
plt.yticks(range(len(indices)), np.array(FEATURES)[indices])
plt.xlabel("Mean decrease in AUC (permutation importance)")
plt.tight_layout()

plt.savefig("Figure_PermutationImportance_primary5_RF.png", dpi=600, bbox_inches="tight")
plt.savefig("Figure_PermutationImportance_primary5_RF.tiff", dpi=600, bbox_inches="tight")
plt.close()

print("✅ Permutation importance figure saved (600 dpi)")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

# ===================== 配置 =====================
MODEL_PKL = "best_model_primary_5_random_forest.pkl"
DEV_XLSX = "your_dataset.xlsx"
TARGET_COL = "group"

FEATURES = [
    'your_data'
]

# ===================== 读取数据 =====================
df = pd.read_excel(DEV_XLSX)
X = df[FEATURES].apply(pd.to_numeric)

# ===================== 加载模型 =====================
model = joblib.load(MODEL_PKL)

# 兼容 Pipeline
if hasattr(model, "named_steps") and ("clf" in model.named_steps):
    clf = model.named_steps["clf"]
else:
    clf = model

importances = clf.feature_importances_
indices = np.argsort(importances)

# ===================== 画图 =====================
plt.figure()
plt.barh(range(len(indices)), importances[indices])
plt.yticks(range(len(indices)), np.array(FEATURES)[indices])
plt.xlabel("Mean decrease in impurity")
plt.tight_layout()

plt.savefig("Figure_MDI_primary5_RF.png", dpi=600, bbox_inches="tight")
plt.savefig("Figure_MDI_primary5_RF.tiff", dpi=600, bbox_inches="tight")
plt.close()

print("✅ MDI importance figure saved (600 dpi)")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import shap
import matplotlib.pyplot as plt

MODEL_PKL = "best_model_primary_5_random_forest.pkl"
DEV_XLSX  = "your_dataset.xlsx"

FEATURES = [
    'your_data'
]

OUT_PNG  = "Figure_SHAP_summary_primary5_RF.png"
OUT_TIFF = "Figure_SHAP_summary_primary5_RF.tiff"

# ---- data ----
df = pd.read_excel(DEV_XLSX)
X = df[FEATURES].copy()
for c in FEATURES:
    X[c] = pd.to_numeric(X[c], errors="coerce")
X = X.dropna().reset_index(drop=True)
print("X shape:", X.shape)

# ---- model ----
model = joblib.load(MODEL_PKL)
clf = model.named_steps["clf"] if hasattr(model, "named_steps") else model

explainer = shap.TreeExplainer(clf)
sv = explainer.shap_values(X)

# 二分类 list -> 取正类
if isinstance(sv, list):
    sv = sv[1]

arr = np.asarray(sv)
print("Raw SHAP ndim/shape:", arr.ndim, arr.shape)

# ===== 关键：把各种返回统一成 (n, p) =====
# Case A: (n, p) -> OK
if arr.ndim == 2:
    shap_2d = arr

# Case B: (n, p, 2) -> 这是“输出维度=2”，取正类通道
elif arr.ndim == 3 and arr.shape[2] == 2 and arr.shape[1] == len(FEATURES):
    shap_2d = arr[:, :, 1]

# Case C: (n, p, p) -> 交互矩阵，聚合成每个特征贡献（main+interaction）
elif arr.ndim == 3 and arr.shape[1] == arr.shape[2] == len(FEATURES):
    shap_2d = arr.sum(axis=2)

else:
    raise ValueError(f"Unrecognized SHAP shape: {arr.shape}")

print("Converted SHAP shape:", shap_2d.shape)

# ---- beeswarm ----
plt.figure()
shap.summary_plot(shap_2d, X, plot_type="dot", show=False)
plt.tight_layout()
plt.savefig(OUT_PNG, dpi=600, bbox_inches="tight")
plt.savefig(OUT_TIFF, dpi=600, bbox_inches="tight")
plt.close()
print(f"✅ Saved: {OUT_PNG} / {OUT_TIFF} (600 dpi)")

# ---- mean(|SHAP|) ----
mean_abs = np.mean(np.abs(shap_2d), axis=0)
rank_df = pd.DataFrame({"Feature": FEATURES, "MeanAbsSHAP": mean_abs}).sort_values(
    "MeanAbsSHAP", ascending=False
).reset_index(drop=True)

print("\n=== SHAP mean(|value|) ranking (class=1) ===")
print(rank_df)

rank_df.to_csv("SHAP_mean_abs_ranking_primary5_RF.csv", index=False, encoding="utf-8-sig")
print("\n✅ Saved: SHAP_mean_abs_ranking_primary5_RF.csv")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold

# ===================== 配置 =====================
MODEL_PKL = "best_model_primary_5_random_forest.pkl"
DEV_XLSX = "your_dataset.xlsx"
TEST_XLSX = "your_dataset.xlsx"
TARGET_COL = "group"
RANDOM_STATE = 42

FEATURES = [
    'your_data'
]

# 风险分层阈值（推荐）
LOW_CUT = 0.20
HIGH_CUT = 0.50

# ===================== 工具函数 =====================
def to_numeric(df, cols):
    X = df[cols].copy()
    for c in cols:
        X[c] = pd.to_numeric(X[c], errors="coerce")
    return X.dropna()

def oof_predict_proba(base_model, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    oof = np.zeros(len(y))
    for tr, va in cv.split(X, y):
        m = clone(base_model)
        m.fit(X.iloc[tr], y[tr])
        oof[va] = m.predict_proba(X.iloc[va])[:, 1]
    return oof

def assign_risk_group(p):
    return np.where(p < LOW_CUT, "Low",
           np.where(p <= HIGH_CUT, "Intermediate", "High"))

def compute_event_rate(y, p):
    groups = assign_risk_group(p)
    df = pd.DataFrame({"y": y, "group": groups})
    return df.groupby("group")["y"].mean()

# ===================== 读取数据 =====================
df_dev = pd.read_excel(DEV_XLSX)
df_te  = pd.read_excel(TEST_XLSX)

y_dev = pd.to_numeric(df_dev[TARGET_COL]).values
y_te  = pd.to_numeric(df_te[TARGET_COL]).values

X_dev = to_numeric(df_dev, FEATURES)
X_te  = to_numeric(df_te, FEATURES)

# 对齐 y（防止 dropna 后错位）
y_dev = y_dev[X_dev.index]
y_te  = y_te[X_te.index]

# ===================== 加载模型 =====================
model = joblib.load(MODEL_PKL)

# Development OOF
p_dev = oof_predict_proba(model, X_dev, y_dev)

# val
final_model = clone(model)
final_model.fit(X_dev, y_dev)
p_te = final_model.predict_proba(X_te)[:, 1]

# ===================== 计算真实发生率 =====================
rate_dev = compute_event_rate(y_dev, p_dev)
rate_te  = compute_event_rate(y_te, p_te)

print("\nDevelopment event rate by risk group:")
print(rate_dev)

print("\nval event rate by risk group:")
print(rate_te)

# ===================== 画图 =====================
labels = ["Low", "Intermediate", "High"]

dev_vals = [rate_dev.get(g, 0) for g in labels]
te_vals  = [rate_te.get(g, 0) for g in labels]

x = np.arange(len(labels))
width = 0.35

plt.figure()
plt.bar(x - width/2, dev_vals, width, label="Development (OOF)")
plt.bar(x + width/2, te_vals, width, label=" validation")

plt.xticks(x, labels)
plt.ylabel("Observed necrotizing pneumonia rate")
plt.xlabel("Predicted risk group")
plt.legend()
plt.tight_layout()

plt.savefig("Figure_RiskStratification_primary5_RF.png", dpi=600, bbox_inches="tight")
plt.savefig("Figure_RiskStratification_primary5_RF.tiff", dpi=600, bbox_inches="tight")
plt.close()

print("✅ Risk stratification figure saved (600 dpi)")
